In [ ]:
import os
import torch
import timm
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
import zipfile
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
zip_path="/content/drive/MyDrive/Research_project/TrashNeXt Dataset.zip"
extract_path = "/content"


In [ ]:
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted to:", extract_path)
!ls {extract_path}

Dataset extracted to: /content
 drive	 __MACOSX   sample_data  'TrashNeXt Dataset'


In [ ]:
import os
from PIL import Image
from tqdm import tqdm

def is_corrupted_image(file_path):
    """Check if an image file is corrupted."""
    try:
        with Image.open(file_path) as img:
            img.verify()  # Verify integrity
        return False
    except (IOError, SyntaxError, Image.DecompressionBombError):
        return True

def remove_corrupted_images(dataset_path):
    """Scan and remove corrupted images + hidden macOS files."""
    corrupted_count = 0
    total_images = 0

    for root, _, files in os.walk(dataset_path):
        for file in tqdm(files, desc=f"Scanning {os.path.basename(root)}"):

            # Skip hidden files (.DS_Store, ._files)
            if file.startswith("."):
                file_path = os.path.join(root, file)
                try:
                    os.remove(file_path)
                    corrupted_count += 1
                    print(f"Removed hidden file: {file_path}")
                except:
                    pass
                continue

            # Check only image formats
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                total_images += 1
                file_path = os.path.join(root, file)
                if is_corrupted_image(file_path):
                    try:
                        os.remove(file_path)
                        corrupted_count += 1
                        print(f"Removed corrupted image: {file_path}")
                    except Exception as e:
                        print(f"Error removing {file_path}: {e}")

    print("\n✅ Scan completed!")
    print(f"📷 Total images scanned: {total_images}")
    print(f"🗑️ Corrupted/hidden files removed: {corrupted_count}")

# Run the cleaner
dataset_path = '/content/dataset'  # Your dataset path
remove_corrupted_images(dataset_path)


Scanning dataset: 0it [00:00, ?it/s]
Scanning Train: 0it [00:00, ?it/s]
Scanning plastic: 100%|██████████| 2135/2135 [00:00<00:00, 7483.87it/s]
Scanning Test: 0it [00:00, ?it/s]
Scanning plastic: 100%|██████████| 267/267 [00:00<00:00, 8238.95it/s]
Scanning Valid: 0it [00:00, ?it/s]
Scanning plastic: 100%|██████████| 267/267 [00:00<00:00, 7011.21it/s]


✅ Scan completed!
📷 Total images scanned: 23625
🗑️ Corrupted/hidden files removed: 0


In [ ]:
from PIL import Image
import os

def remove_truncated_images(dataset_path):
    removed_count = 0  # Counter for removed images

    for root, _, files in os.walk(dataset_path):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(root, file)
                try:
                    img = Image.open(img_path)
                    img.verify()  # Verify if the image is not corrupted
                    img.close()   # Close the image to avoid resource leaks
                except Exception as e:
                    print(f"Removing corrupted/truncated: {img_path}")
                    os.remove(img_path)
                    removed_count += 1

    print(f"\nTotal corrupted/truncated images removed: {removed_count}")

# Example usage
dataset_path = "/content/dataset"
remove_truncated_images(dataset_path)


Total corrupted/truncated images removed: 0


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# 🔹 Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
import os
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 🔹 Paths
train_dir = "/content/dataset/Train"
valid_dir = "/content/dataset/Valid"
test_dir  = "/content/dataset/Test"

# 🔹 Transforms (CoAtNet standard: 224x224)
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

valid_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
# 🔹 Datasets & Loaders
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_dataset = datasets.ImageFolder(valid_dir, transform=valid_transforms)
test_dataset  = datasets.ImageFolder(test_dir,  transform=valid_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False)

# Print class names
print(f"Classes: {train_dataset.classes}")
print(f"Number of classes: {len(train_dataset.classes)}")

Classes: ['cardboard', 'e-waste', 'foam_rubber', 'glass', 'medical', 'metal', 'organic', 'paper', 'plastic']
Number of classes: 9


In [ ]:

num_classes = len(train_dataset.classes)
model = timm.create_model("coatnet_0_rw_224", pretrained=True, num_classes=num_classes)
model.to(device)


MaxxVit(
  (stem): Stem(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (norm1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (stages): Sequential(
    (0): MaxxVitStage(
      (blocks): Sequential(
        (0): MbConvBlock(
          (shortcut): Downsample2d(
            (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
            (expand): Conv2d(64, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
          (pre_norm): BatchNormAct2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (down): Downsample2d(
            (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
            (expand): Identity

In [ ]:
# 🔹 Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4)

In [ ]:
# 🔹 Training Loop
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} - Training"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 🔹 Track loss & accuracy
        running_loss += loss.item()
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

    train_acc = 100. * correct / total

    # 🔹 Validation
    model.eval()
    val_correct, val_total = 0, 0
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = outputs.max(1)
            val_correct += preds.eq(labels).sum().item()
            val_total += labels.size(0)

    val_acc = 100. * val_correct / val_total
    print(f"\nEpoch [{epoch+1}/{epochs}] "
          f"Train Loss: {running_loss/len(train_loader):.4f} | "
          f"Train Acc: {train_acc:.2f}% | "
          f"Valid Loss: {val_loss/len(valid_loader):.4f} | "
          f"Valid Acc: {val_acc:.2f}%")


Epoch 1 - Training:   2%|▏         | 13/591 [00:15<09:26,  1.02it/s]/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Epoch 1 - Training: 100%|██████████| 591/591 [10:58<00:00,  1.11s/it]



Epoch [1/5] Train Loss: 0.8430 | Train Acc: 72.21% | Valid Loss: 0.5593 | Valid Acc: 81.93%


Epoch 2 - Training: 100%|██████████| 591/591 [10:44<00:00,  1.09s/it]



Epoch [2/5] Train Loss: 0.5221 | Train Acc: 83.06% | Valid Loss: 0.5366 | Valid Acc: 83.16%


Epoch 3 - Training: 100%|██████████| 591/591 [10:42<00:00,  1.09s/it]



Epoch [3/5] Train Loss: 0.4147 | Train Acc: 86.40% | Valid Loss: 0.4386 | Valid Acc: 85.87%


Epoch 4 - Training: 100%|██████████| 591/591 [10:41<00:00,  1.09s/it]
